In [2]:
import pandas as pd
import numpy as np
import seaborn as sb

data = pd.read_pickle('Binning.pickle')

data


,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot,JOININGYEAR_BIN,AGE_BIN,EXPERIENCEINCURRENTDOMAIN_BIN
1,Bachelors,2013,C,1,28,Female,No,3,1,"(2011.999, 2013.0]","(27.0, 28.0]","(2.0, 3.0]"
2,Bachelors,2014,B,3,38,Female,No,2,0,"(2013.0, 2014.0]","(34.0, 38.0]","(1.0, 2.0]"
3,Masters,2016,A,3,27,Male,No,5,1,"(2015.0, 2016.0]","(26.0, 27.0]","(4.0, 5.0]"
4,Masters,2017,C,3,24,Male,Yes,2,1,"(2016.0, 2017.0]","(21.999, 24.0]","(1.0, 2.0]"
6,Bachelors,2015,B,3,38,Male,No,0,0,"(2014.0, 2015.0]","(34.0, 38.0]","(-0.001, 1.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...
4647,Bachelors,2016,C,3,30,Male,No,2,0,"(2015.0, 2016.0]","(29.0, 31.0]","(1.0, 2.0]"
4648,Bachelors,2013,A,3,26,Female,No,4,0,"(2011.999, 2013.0]","(24.0, 26.0]","(3.0, 4.0]"
4649,Masters,2013,C,2,37,Male,No,2,1,"(2011.999, 2013.0]","(34.0, 38.0]","(1.0, 2.0]"
4650,Masters,2018,B,3,27,Male,No,5,1,"(2017.0, 2018.0]","(26.0, 27.0]","(4.0, 5.0]"


In [4]:
def num_event(event, dataframe ,field, val):
    if event == 'good':
        event = 1
    else:
        event = 0
    cond_main = dataframe[field] == val
    cond_event = dataframe['LeaveOrNot'] == event
    n = len(dataframe[cond_main & cond_event].index)
    return n


def dist_event(event, dataframe ,field, val):
    n = num_event(event, dataframe, field, val)
    sumd = 0
    all_elements = set(dataframe[field].to_numpy())
    for a in all_elements:
        s = num_event(event, dataframe, field, a)
        sumd = sumd + s
    return n/sumd

def WOE(dataframe, field, val):
    dg = dist_event('good', dataframe, field, val)
    db = dist_event('bad', dataframe, field, val)
    w = np.log(dg / db)
    return w
dt_train = pd.DataFrame()
dt_train['LeaveOrNot'] = data['LeaveOrNot']
ATTRS = ['Education', 'PaymentTier', 'Age', 'Gender', 'EverBenched','JOININGYEAR_BIN','EXPERIENCEINCURRENTDOMAIN_BIN']


NEW_WOE_ATTRS=[]
for attr in ATTRS:
    attr_element = set(data[attr].to_numpy())
    attr_woe = dict()
    for i in attr_element:
        w = WOE(data, attr, i)
        attr_woe[i] = w
        NEW_WOE_ATTRS.append( attr_woe[i])
    print(f'WOE {attr} -> {attr_woe}')
    

    lst = []
    for s in data[attr]:
        woe = attr_woe[s]
        lst.append(woe)

    new_field = attr + '_WOE'
    dt_train[new_field] = lst


dt_train



WOE Education -> {'PHD': -0.45016406746450516, 'Bachelors': -0.1513965079384955, 'Masters': 0.6239421825883269}
WOE PaymentTier -> {1: 0.08452503055593452, 2: 1.0395191516220397, 3: -0.3151325750622815}
WOE Age -> {22: 0.07102434333703207, 23: 0.016957122066756293, 24: 0.11648671741378933, 25: 0.2951430288094481, 26: 0.12388783289409973, 27: 0.04275390939877661, 28: -0.2147903149399535, 29: 0.05581492467350306, 30: -0.13752040855602504, 31: -0.2648598309326267, 32: 0.4356674569249413, 33: -0.16786756494531696, 34: -0.1635479038008005, 35: 0.22517502316429028, 36: -0.2716643586269606, 37: -0.09685652930212822, 38: -0.1986392236120705, 39: -0.18386790629175795, 40: -0.2284921867617519, 41: -0.12964635212511913}
WOE Gender -> {'Female': 0.5205543885381253, 'Male': -0.39793155910410033}
WOE EverBenched -> {'Yes': 0.48627745781135934, 'No': -0.060124089222305874}
WOE JOININGYEAR_BIN -> {Interval(2017.0, 2018.0, closed='right'): 4.769684872412462, Interval(2014.0, 2015.0, closed='right'): 0.

,LeaveOrNot,Education_WOE,PaymentTier_WOE,Age_WOE,Gender_WOE,EverBenched_WOE,JOININGYEAR_BIN_WOE,EXPERIENCEINCURRENTDOMAIN_BIN_WOE
1,1,-0.151397,0.084525,-0.214790,0.520554,-0.060124,-0.240045,0.096223
2,0,-0.151397,-0.315133,-0.198639,0.520554,-0.060124,-0.555915,0.049123
3,1,0.623942,-0.315133,0.042754,-0.397932,-0.060124,-0.528150,-0.151566
4,1,0.623942,-0.315133,0.116487,-0.397932,0.486277,-0.355434,0.049123
6,0,-0.151397,-0.315133,-0.198639,-0.397932,-0.060124,0.265322,0.036616
...,...,...,...,...,...,...,...,...
4647,0,-0.151397,-0.315133,-0.137520,-0.397932,-0.060124,-0.528150,0.049123
4648,0,-0.151397,-0.315133,0.123888,0.520554,-0.060124,-0.240045,-0.024946
4649,1,0.623942,1.039519,-0.096857,-0.397932,-0.060124,-0.240045,0.049123
4650,1,0.623942,-0.315133,0.042754,-0.397932,-0.060124,4.769685,-0.151566


In [5]:
Y =  dt_train['LeaveOrNot'].to_numpy()
x_1 = dt_train['Education_WOE'].to_numpy()
x_2 = dt_train['PaymentTier_WOE'].to_numpy()
x_3 = dt_train['Age_WOE'].to_numpy()
x_4 = dt_train['Gender_WOE'].to_numpy()
x_5 = dt_train['EverBenched_WOE'].to_numpy()
x_6 = dt_train['JOININGYEAR_BIN_WOE'].to_numpy()
x_7 = dt_train['EXPERIENCEINCURRENTDOMAIN_BIN_WOE'].to_numpy()



array_ones = np.ones(x_1.shape)

X = np.array([array_ones, x_1,x_2,x_3,x_4,x_5,x_6,x_7])
X = np.transpose(X)
X


array([[ 1.        , -0.15139651,  0.08452503, ..., -0.06012409,
        -0.24004466,  0.09622345],
       [ 1.        , -0.15139651, -0.31513258, ..., -0.06012409,
        -0.55591459,  0.0491228 ],
       [ 1.        ,  0.62394218, -0.31513258, ..., -0.06012409,
        -0.52815045, -0.15156626],
       ...,
       [ 1.        ,  0.62394218,  1.03951915, ..., -0.06012409,
        -0.24004466,  0.0491228 ],
       [ 1.        ,  0.62394218, -0.31513258, ..., -0.06012409,
         4.76968487, -0.15156626],
       [ 1.        , -0.15139651, -0.31513258, ...,  0.48627746,
        -0.24004466,  0.0491228 ]])

In [6]:
np.matmul(np.transpose(X), X)


array([[ 3.25800000e+03, -4.25521243e+01, -9.80892534e+01,
        -1.77614667e+01, -9.87553438e+01, -1.17469613e+01,
         3.96545327e+02, -3.80598313e+00],
       [-4.25521243e+01,  3.29048694e+02,  1.38263183e+02,
         6.65349205e-01,  8.90148525e+00,  1.04388055e+00,
         2.55602061e+00,  2.78181374e+00],
       [-9.80892534e+01,  1.38263183e+02,  9.34772803e+02,
        -1.62667346e+00,  2.13110162e+02,  1.25958660e+00,
        -2.41993860e+02,  2.77681197e+00],
       [-1.77614667e+01,  6.65349205e-01, -1.62667346e+00,
         1.14599974e+02, -4.89468170e+00,  7.57899910e-01,
         2.00973505e+01,  1.21061657e+00],
       [-9.87553438e+01,  8.90148525e+00,  2.13110162e+02,
        -4.89468170e+00,  6.62768814e+02, -5.10557057e+00,
        -1.72359399e+01,  6.58475880e-01],
       [-1.17469613e+01,  1.04388055e+00,  1.25958660e+00,
         7.57899910e-01, -5.10557057e+00,  9.02481039e+01,
         8.69945095e+01, -7.16817946e-01],
       [ 3.96545327e+02,  2.556020

In [7]:
np.linalg.inv(np.matmul(np.transpose(X), X))


array([[ 3.12150750e-04,  3.33460512e-05,  1.26311272e-05,
         5.28891389e-05,  4.22813557e-05,  6.25625214e-05,
        -2.08989928e-05,  5.04479980e-05],
       [ 3.33460512e-05,  3.25802687e-03, -5.12085486e-04,
        -8.25154783e-06,  1.25527272e-04,  7.77941702e-07,
        -2.31562556e-05, -3.16874641e-04],
       [ 1.26311272e-05, -5.12085486e-04,  1.24942027e-03,
        -1.21642356e-06, -3.92237774e-04, -8.03298654e-05,
         4.93144090e-05, -9.35243262e-05],
       [ 5.28891389e-05, -8.25154783e-06, -1.21642356e-06,
         8.74762830e-03,  7.23331357e-05, -3.58166544e-05,
        -3.11436713e-05, -4.36123686e-04],
       [ 4.22813557e-05,  1.25527272e-04, -3.92237774e-04,
         7.23331357e-05,  1.64058684e-03,  1.16833686e-04,
        -1.56885687e-05, -2.52601621e-06],
       [ 6.25625214e-05,  7.77941702e-07, -8.03298654e-05,
        -3.58166544e-05,  1.16833686e-04,  1.12626255e-02,
        -1.68630699e-04,  4.20903607e-04],
       [-2.08989928e-05, -2.315625

In [8]:
np.matmul(np.transpose(X), Y)


array([1111.        ,   62.59291184,  186.48085465,   19.74867017,
        115.41900805,   17.34797512,  986.14194406,    3.88538624])

In [9]:
B = np.matmul(np.linalg.inv(np.matmul(np.transpose(X), X)), np.matmul(np.transpose(X), Y))
B

array([0.3378387 , 0.13575495, 0.21655157, 0.20609121, 0.15901611,
       0.09807848, 0.14702715, 0.11746988])

In [10]:
def predict(x):
    y = B[0] + B[1]*x+ B[2]*x+ B[3]*x+ B[4]*x+ B[5]*x \
    + B[6]*x+ B[7]*x
    return y

eq = f'y = {B[0]:.3f}'
for i in range(1, len(B)):
    eq = eq + f'+ ({B[i]:.3f})x_{i}'
print(eq)

y = 0.338+ (0.136)x_1+ (0.217)x_2+ (0.206)x_3+ (0.159)x_4+ (0.098)x_5+ (0.147)x_6+ (0.117)x_7


In [11]:
data_test = pd.read_pickle('Data_test.pickle')

attr = pd.read_pickle('attr.pickle')
attr


,ATTR,VAL,N_GOOD,N_BAD,DIST_GOOD,DIST_BAD,WOE
0,Age,22,10,18,0.009001,0.008384,0.071024
1,Age,23,10,19,0.009001,0.008850,0.016957
2,Age,24,100,172,0.090009,0.080112,0.116487
3,Age,25,114,164,0.102610,0.076386,0.295143
4,Age,26,164,280,0.147615,0.130415,0.123888
5,Age,27,155,287,0.139514,0.133675,0.042754
6,Age,28,134,321,0.120612,0.149511,-0.214790
7,Age,29,58,106,0.052205,0.049371,0.055815
8,Age,30,46,102,0.041404,0.047508,-0.137520
9,Age,31,27,68,0.024302,0.031672,-0.264860


In [12]:
# y_predict
data_test['y_predict'] = B[0] + B[1]*dt_train['Education_WOE'] + B[2]*dt_train['PaymentTier_WOE'] \
        + B[3]*dt_train['Age_WOE'] + B[4]*dt_train['Gender_WOE'] + B[5]*dt_train['EverBenched_WOE'] \
        + B[6]*dt_train['JOININGYEAR_BIN_WOE'] + B[7]*dt_train['EXPERIENCEINCURRENTDOMAIN_BIN_WOE']

data_test['P'] = 1/(1+np.exp(-data_test['y_predict']))

data_test['predict'] = data_test['P'].map(lambda x: 1 if x >= 0.5 else 0)
data_test

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot,y_predict,P,predict
0,Bachelors,2017,A,3,34,Male,No,0,0,NaN,NaN,0
5,Bachelors,2016,A,3,22,Male,No,0,0,NaN,NaN,0
7,Bachelors,2016,A,3,34,Female,No,2,1,NaN,NaN,0
13,Bachelors,2016,A,3,39,Male,No,2,0,NaN,NaN,0
15,Bachelors,2017,A,1,29,Male,No,3,0,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4639,Bachelors,2017,A,3,28,Male,No,0,1,NaN,NaN,0
4640,Bachelors,2015,A,3,35,Male,No,0,0,NaN,NaN,0
4644,Bachelors,2015,C,3,32,Female,Yes,1,1,NaN,NaN,0
4646,Bachelors,2013,A,3,25,Female,No,3,0,NaN,NaN,0


In [13]:
n_correct = (data_test['LeaveOrNot'] == data_test['predict']).sum()
print(n_correct)

accuracy = n_correct/1395
print('accuracy is')
print(accuracy)


906
accuracy is
0.6494623655913978
